In [63]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [64]:
!ls

detect-57-ensemble.ipynb
detect-57-ensemble-level2.ipynb
eda.ipynb
filter_detect_submission.ipynb
filter_every_relation.ipynb
filter_rel_class.ipynb
Is-classifier-6-eda.ipynb
Is-detect-42-eda.ipynb
Is-detector-result.ipynb
lb18901
lb19252
lb26760
merge_0905_1_plays_holds_insideof_on.csv
merge_0905_1_plays_holds_insideof_on_top30.csv.zip
merge_0905_1_plays_holds_insideof_on_top50.csv.zip
merge_df.ipynb
show-detections.ipynb
submit-detect-57.ipynb
submit-is-42-ensemble.ipynb
submit-is-42.ipynb
submit-is-6-catboost.ipynb
submit-is-6.ipynb
tmp.csv
top_classes_42.csv
top_classes.csv
train_lgbm.ipynb
tuple-rel-classifier.ipynb
tuple-rel-classifier-new.ipynb
tuple-rel-classifier-new-lb18901.ipynb


In [65]:
df = pd.read_csv('merge_0905_1_plays_holds_insideof_on.csv')
df.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.387808 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.101016 /m/04yx4 0.9498244 0.3313462 0.9990...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.168493 /m/01_5g 0.7968998 0.0202855 0.99...


In [66]:
df.PredictionString = df.PredictionString.fillna('')

In [67]:
from collections import defaultdict
TOPK = 80
def get_filtered_pred_string(pred_str):
    rel_dict = defaultdict(list)
    if len(pred_str) < 1:
        return ''
    det = []
    det_items = pred_str.split(' ')
    det_items = [x for x in det_items if len(x) > 0]
    for i, e in enumerate(det_items):
        if i % 12 == 0:
            det = []
        det.append(e)
        if (i+1) % 12 == 0:
            #print(i, det[11])
            rel_dict[det[11]].append(det)
    res = []
    for rel in rel_dict:
        filtered_dets = sorted(rel_dict[rel], key=lambda x: x[0], reverse=True)[:TOPK]
        for fd in filtered_dets:
            res.append(' '.join(fd))
    #print(rel_dict)
    return ' '.join(res)

In [68]:
df['PredictionString'] = df.PredictionString.map(lambda x: get_filtered_pred_string(x))
df.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.387808 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.101016 /m/04yx4 0.9498244 0.3313462 0.999023...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.168493 /m/01_5g 0.7968998 0.0202855 0.997210...


In [69]:
df.to_csv('merge_0905_1_plays_holds_insideof_on_top80.csv.zip', columns=['ImageId', 'PredictionString'], index=False, compression='zip')

In [70]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f merge_0905_1_plays_holds_insideof_on_top80.csv.zip -m 'top 80 for each relation'

100%|████████████████████████████████████████| 381M/381M [00:09<00:00, 42.0MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship